In [87]:
from torch.utils.data import Dataset, DataLoader
import json
import gensim.models as g
import jieba
import re
import torch
import numpy as np  
from torch.nn.utils.rnn import pack_padded_sequence,pad_sequence
from torchtext.vocab import Vectors
jieba.load_userdict("dict.txt")
stop_word_list = ["我们","大家","一起","你们","这个","一定","没有","现在","可以","一个","做好","怎么","就是","什么" ,"一直","还是","不是","这么","已经","今天","一下","好好","起来","一切","若有","岁月静好","或者","到底","一列","能够","zf","各种","好多","人们","最大","一句","一天","人数","值得","是因为","蔡","一个月","包括","一种","并且","千万","卧槽","少去",\
"前往","一幕","致","个别","主要","别的","一群","哪个","华晨","一边","一家","一场","团肖战","它们","至少","非要","是从","今日","吕贝克","尽量少","年月日时",
"不停","某些","不停","干嘛","唯一","一遍","人心","如今","致以","应该","身后","一些","您们","进群","有",'都','我','不','要', '人', '你', '好', '心', '有',
'就', '和', '愿', '例', '向', '能', '还', '去', '戴', '到', '会', '们', '给', '多', '吃', '被', '为', '看', '很', '吗', '让', '对', '战', '（', '他',
'等', '）', '来', '上', '想', '但', '没', '请', '做', '过', '太', '年', '泪', '从', '又', '最', '跪', '用', '着', '爱', '与', '把', '死', '再', '更',
'一', '月', '得', '大', '后', '日', '并', '她', '才', '赞', '那', '逼', '跟', '号', '已', '比', '而', '快', '带', '急', '可', '中', '个', '里', '地',
'买', '别', '呗', '妈', '新', '无', '小', '啥', '花', '谁', '跑', '打', '喵', '该', '哦', '下', '或', '怒', '只', '时', '挺', '事', '傻', '呀', '出',
'天', '哭', '前', '完', '将', '怕', '者', '么', '听', '点', '每', '岁', '真', '祝', '您', '名', '求', '走', '它', '却', '少', '如', '住', '像', '搞',
'转', '慌', '叫', '同', '高', '站', '\xa0', '起', '啦', '连', '热', '及', '病', '省', '由', '当', '万', '搜', '内', '性', '吐', 'N', '于', 'G', '香',
 '称', '劝', '家', '往', '拿', '先', '逆', '之', '发', '回', '牛', '东', '靠', '长', '以', '找', '正', '问', '刷', '总', '均', '因', '嗯', '封', '刚',
 '写', '哪', '至', '榜', '咋', '越', '嘛', '→', '早', '蔡徐坤', '钱', '唉', '棒', '呆', '卖', '倍', '喝', '各', '放', '命', '|', '杨', '元', '算', '哎',
  '防', '区', '忙', '干', '源', '则', '顶', '离', '允', '卫', '见', '汗', '成', '西', '市', '手', '↓', '孙', '另', '懂', '嗒', '机', '链', '贝', '难',
  '憨', '贺', '①', '②', '③', '瓜', '抓', '话', '路', '冲', '晕', '镜', '耶', '未', '全', '爸', '退', '开', '自', '脸', '疯', '所', '应', '待', '望',
  '其', '且', '服', '换', '酷', '赴', '抄', '哼', '行', '穿', '玩', '讲', '划', '宇', '经', 'X', '弱', '哈', '红', '史', '口', '治', '衰', '敢', '副',
   '线', '嫌', '瞎', '进', '玖', '黑', '字', '鼻', '北', '谭', '碰', '哇', '城', '挤', '头', '乱', '曾', '远', '委', '操', '臭', '四', '停', '勤',
   '台', '王', '抗', '尽', '门', '约', '力', '按', '共', '本', '倒', '獾', '嘴', '传', '受', '感', '④', '℃', '摘', '微', '老', '替', '某', '差',
   '啵', '达', '批', '⑤', 'C', '图', '位', '杀', '须', '据', '信', '既', '仍', '們', '噢', '蒸', '使', '此', '气', '留', '档', '猪', '滚', 'n',
   '砍', '非', '活', '囧', '药', '侠', '钟', '草', '票', '衣', '捂', '救', '错', '狗', '定', '怪', '抢', '哒', '三', '二', '处', '群', '捐',
   '零', '忘', '期', '即', '绝', '洗', '谈', '拉', '烊', '作', '冒', '轮', '害', '肺', '掉', '左', '强', '毒', 'V', '崽', '涨', '数', '遭',
   '书', '易', '车', '报', '管', '晚', '蛮', '套', '喜', '拍', '近', '仗', 'p', '锅', '第', '展', '学', '需', '语', '坐', '鸭', '女', '說',
   '弄', '条', '够', '马', '疼', '坏', '闹', '喷', '些', '揉', '山', '呜', '件', '佛', '累', '村', '养', '特', '’', '建', '仅', '饿', '肉',
   '变', '诶', '赚', '外', '赢', '辈', '捡', '霾', '科', '伴', '烦', '若', '屎', '扔', '贴', '查', '拖', '七', '嘘', '喊', '困', '咱', '收'
   , '端', '亲', '县', '盼', '熬', '陪', '囤', '党', '含', '挂', '醋', '爆', '原', '蛇', '道', '蹲', '民', '港', '加', 'b', '丢', '度', '×',
   '祎', '米', '伤', '羞', '负', '接', '假', '播', '屁', '现', '次', '浪', '瞒', '镇', '办', '凭', '量', '飞', '照', '蹭', '浙', '分', '边',
   '存', '较', '借', '水', '］', '扛', '［', '醒', '藏', '官', '尚', '呈', '网', 'ω', '味', '盒', '生', '实', '矿', '店', '便', '厅', '补',
   '妳', '轻', '惹', '咬', '猫', '俩', '剧', '逛', '云', '犯', '鼠', '装', '贱', '烧', '翻', '贵', '願', '‘', '吹', '纸', '爬', '局', '广',
   '轶', '戳', '废', '恨', '低', '贰', '删', '控', '骗', '领', '躺', '吓', '兹', '鬼', '汤', '埃', '货', '哟', '踩', '反', '权', 'g', '何',
   '指', '炸', '亏', '身', '包', '蠢', '鸡', '继', '推', '岗', '相', '徐', '赌', '腰', '剩', '叭', '抽', '甜', '吖', '俺', '滴', '逃', '﹏',
   '⑥', '张', '重', '患', 'e', '周', '九', '福', '版', '阿', '菜', '慢', '粉', '神', '级', '任', '背', '＾', '压', '壶', '页', 'T', '吨',
   '缺', '酒', '间', '久', 'w', '法', '憋', '蛊', '予', '儿', '读', "'", '令', '超', '止', '谣', '五', '改', '遍', '唐', '扣', '吼', '♂',
   '怂', '皮', '宠', '李', '薇', '娅', '撒', '跳', '枪', '皆', '乡', '造', '脚', '籍', '撤', '谱', 'D', '邪', '主', '熟', '半', '為', '粑',
   '砸', '项', '卻', '潮', '排', '嗨', '旻', '抠', '呕', '附', '安', '友', '摊', '追', '饭', '化', '宁', 'x', '班', '举', '哥', '朱', '骑',
   '夹', '抖', '呃', '双', '挡', '迟', '陶', '撞', '物', '面', '梗', '核', '沪', '守', '组', '拟', '陈', '余', '罪', '輩', '長', '男', '散',
   '苦', '醉', '雾', '奔', '探', '寒', '喘', '幺', '财', '恶', '言', '满', '考', '岛', '敌', 'c', '雨', '抱', '眼', '–', '°', '脑', '型', 'A',
    '炖', '园', '揪', 'o', '宝', '风', '虽', '床', '淚', '占', '跨', '揍', '块', '摆', '昙', '舞', '嘎', '躲', '泡', '潭', '方', '竟', '具', '投',
     '己', '深', '拼', '窝', '淦', '坎', '黄', '夸', '美', '痰', '短', '两', '蛆', '除','）', '来', '上', '想', '但', '没', '请', '做', '过', '太',
     '年', '泪', "ω","⑤","④","有","就","和","是", "的" ,"因为", "了", "也","在","波","啊","呢","吧","nf","每到","这场","这","例","哦","嘛","吗"]

def read_stop_word():
    with open("stop_word.txt","r",encoding="utf-8") as f:
        stop_word_dic = f.read()
    for word in stop_word_dic:
        if word == "\n" :
            continue
        stop_word_list.append(word)

def read_data():
    x = []
    y = []
    y_map = {"happy":0 , "sad" :1 , "angry" : 2 , "neural" : 3, "fear" : 4 , "surprise" : 5}
    with open("virus_eval_labeled.txt","r",encoding="utf-8") as f: 
        data = f.read()
    data = json.loads(data)
    for i in data:
        s = re.sub("//@.*?:","", i["content"])
        s = re.sub("(ht|f)tp(s?)\:\/\/[0-9a-zA-Z]([-.\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%$#_]*)?","",s)
        s = re.sub("[\[\]_.!！+-=–—,$%·^『』～；\"，“”。?？＋、：V％~@#￥%……&*《》<>「」{}【】()/]","",s)
        s.replace("|","").replace("（","").replace("–","").replace("°","").replace("o","")
        if s is None and s == " ":
            continue
        segment_list = jieba.cut(s,cut_all=False)
        # segment = ",".join(segment_list)
        # print(segment.split(","))
        temp_list = []
        for word in segment_list:
            if word not in stop_word_list:
                temp_list.append(word)
        if not temp_list or len(temp_list) == 0:
            continue
        x.append(temp_list)
        y.append(y_map[i["label"]])
    return  x , y
x , y = read_data()
print(len(x),len(y))


1941 1941


In [11]:
# X = []
# Y = []
# print(len(x),len(y))
# for i in range(len(x)):
#     if not x[i]:
#         continue
#     X.append(x[i])
#     Y.append(y[i])
# print(len(X),len(Y))

    
    


8606 8606
8363 8363


In [88]:
word2vec = g.Word2Vec(x, 
                       vector_size=64,   # 词向量维度
                       min_count=3,      # 最小词频, 因为数据量较小, 这里卡1
                       epochs=100) 
word2vec.save("word2vec.model")
model = g.Word2Vec.load("word2vec.model")
model.train([["hello", "world"]], total_examples=1, epochs=1)
# print(word2vec.wv.most_similar("武汉"))
# vectors = Vectors(name='./weibo.vector.txt')

In [14]:
word2vec.wv.most_similar("天使")
# word2vec.wv.most_similar("天使")
# for i ,v in word2vec.wv.key_to_index.items():
#     print(i,v  )
# word2vec.wv["的"]


[('敬佩', 0.6305860877037048),
 ('救死扶伤', 0.6112337708473206),
 ('感恩', 0.5958109498023987),
 ('勇敢', 0.5928530693054199),
 ('坚守', 0.5810125470161438),
 ('阳光', 0.5683488845825195),
 ('逆行', 0.5573121905326843),
 ('国难', 0.5476272106170654),
 ('生命危险', 0.5454411506652832),
 ('道德', 0.5237178206443787)]

In [89]:
# 
data_list = []
label = []
count = 0
for s in x:
    vectors = []
    for w in s:
        if w in word2vec.wv.key_to_index:
            vectors.append(word2vec.wv[w])   # 将每个词替换为对应的词向量
            
    if len(vectors) == 0:
        count += 1
        continue
    vectors = torch.Tensor(vectors)
    data_list.append(vectors)
    label.append(y[count])
    count += 1


In [67]:
print(len(data_list))
print(len(label))

for i in data_list:
    print(i.shape)




8249
8249
torch.Size([1, 64])
torch.Size([8, 64])
torch.Size([3, 64])
torch.Size([40, 64])
torch.Size([4, 64])
torch.Size([3, 64])
torch.Size([3, 64])
torch.Size([12, 64])
torch.Size([6, 64])
torch.Size([6, 64])
torch.Size([3, 64])
torch.Size([3, 64])
torch.Size([36, 64])
torch.Size([2, 64])
torch.Size([13, 64])
torch.Size([17, 64])
torch.Size([8, 64])
torch.Size([16, 64])
torch.Size([1, 64])
torch.Size([2, 64])
torch.Size([1, 64])
torch.Size([2, 64])
torch.Size([7, 64])
torch.Size([8, 64])
torch.Size([9, 64])
torch.Size([3, 64])
torch.Size([4, 64])
torch.Size([22, 64])
torch.Size([5, 64])
torch.Size([1, 64])
torch.Size([10, 64])
torch.Size([24, 64])
torch.Size([14, 64])
torch.Size([2, 64])
torch.Size([3, 64])
torch.Size([42, 64])
torch.Size([36, 64])
torch.Size([2, 64])
torch.Size([22, 64])
torch.Size([13, 64])
torch.Size([57, 64])
torch.Size([23, 64])
torch.Size([16, 64])
torch.Size([31, 64])
torch.Size([2, 64])
torch.Size([1, 64])
torch.Size([8, 64])
torch.Size([2, 64])
torch.Size([

In [90]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = data_list
        self.label = label
        
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

def collate_fn(data):
    """
    :param data: 第0维：data，第1维：label
    :return: 序列化的data、记录实际长度的序列、以及label列表
    """
    data.sort(key=lambda x: len(x[0]), reverse=True) # pack_padded_sequence要求要按照序列的长度倒序排列
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # 用RNN处理变长序列的必要操作
    return data, torch.tensor(y, dtype=torch.long), data_length

In [91]:
train_data = MyDataset()
train_loader = DataLoader(train_data, batch_size=32, collate_fn=collate_fn, shuffle=True)


In [10]:
p = []
for i in word2vec.wv.key_to_index:
    print(i)

 
武汉
加油
疫情
致敬
我们
医护人员
肺炎
大家
平安
一起
自己
前线
你们
新型
确诊
新型冠状病毒
希望
口罩
健康
说
真的
病例
保护
一定
辛苦
打赢
没有
感染
这个
一线
防护
戴口罩
现在
可以
病毒
一个
医院
全国
中国
不要
做好
怎么
就是
防疫战
什么
他们
所有
医生
隔离
战斗
出门
已经
注意
封城
一直
知道
微笑
防控
患者
不是
湖北
野味
还是
平平安安
英雄
工作
今天
伟大
不能
这么
拳头
时候
政府
所有人
措施
感谢
工作者
人民
如果
可能
这样
作揖
国家
这次
看到
这种
一下
好好
相信
心心心
死亡
医护
转发
传播
起来
情况
悲伤
支持
泪泪
发热
需要
地方
支援
上海
过年
但是
SARS
防御战
咱们
觉得
很多
哈哈哈
物资
这些
家人
小心
北京
非典
严重
感冒
问题
发现
还有
出来
注意安全
发布
重视
出现
医疗
新闻
新增
安徽
病人
湖北省
为什么
白条
目前
医务人员
过去
春节
超话
京东
勤洗手
预防
野生动物
对不起
开始
话筒
不会
喜欢
传染
照顾
信息
冠状病毒
山东
症状
首例
祝福
防范
费解
害怕
抱抱
挺住
春晚
控制
所以
摊手
允悲
朋友
谢谢
记得
那些
及时
那么
城市
宣传
回家
治疗
一切
别人
直接
出去
武汉市
时间
泪泪泪
这是
人传人
失望
其他
鲜花
一样
回来
感觉
期间
第一
有人
微博
抗击
看看
恐慌
接触
一级
取现
钟南山
咳嗽
有效
取消
可怕
社会
爆发
一点
真是
河南
相关
家里
消息
重要
响应
建议
人员
人类
一急
专家
最后
为了
个人
可爱
傻逼
good
传染病
广东
卫健委
第一线
只有
在家
白衣天使
拜拜
视频
报道
国内
关注
几个
安全
永远
作为
卫生
每天
特别
只是
之后
成功
江苏
多少
身边
生命
市场
几天
完成
来自
恳请
以及
开心
生活
帮助
健健康康
领导
组织
归来
非常
如何
最新
有点
众志成城
官方
骂
关于
病情
嘻嘻
疑似
他人
发生
必须
确认
人群
疾病
昨天
自我
之前
小时
只能
付出
能力
行为
医用
各地
同时
笑
表示
虽然
肯定
新冠
通报
身体
外出
得到
市民
一次
报告
然后
央视
终于
正常
酒精
实时
尽量
普通
蝙蝠
救治
那个


In [80]:

# 网络结构
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = torch.nn.Linear(hidden_size * 2, 6)  # 双向, 输出维度要*2
        # self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)# 双向, 第一个维度要*2
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)
        
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
        packed_out, (h_n, h_c) = self.lstm(packed_input, (h0, c0))

        lstm_out = torch.cat([h_n[-2], h_n[-1]], 1)  # 双向, 所以要将最后两维拼接, 得到的就是最后一个time step的输出
        out = self.fc(lstm_out)
        # out = self.sigmoid(out)
        return out

# lstm = LSTM

In [81]:
# 定义损失函数和优化器
# 每个词向量的维度
embed_size = 64
hidden_size = 64
learning_rate = 5e-4
lstm = LSTM(embed_size,hidden_size,1)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [84]:
# out,hidden = lstm(x[0],len(x[0]))
# 迭代训练
for epoch in range(5):
    total_loss = 0
    for i,data in enumerate(train_loader):
        x, y, lengths = data
        # print(len(x))
#         # x = x.to(device)
        # labels = labels.to(device)
        y_hat = lstm(x, lengths)          # 前向传播
        # print(outputs)
        # print(inputs)

        # logits = outputs.view(-1)           # 将输出展平
        # print(y_hat)
        # print(labels)
        loss = criterion(y_hat, y)    # loss计算
        total_loss += loss
        optimizer.zero_grad()               # 梯度清零
        loss.backward(retain_graph=True)    # 反向传播，计算梯度
        optimizer.step()                    # 梯度更新
        if (i+1) % 10 == 0:
            print("epoch:{}, step:{}, loss:{}".format(epoch+1, i+1, total_loss/10))
            total_loss = 0
        # break

epoch:1, step:10, loss:0.7233811616897583
epoch:1, step:20, loss:0.6330550312995911
epoch:1, step:30, loss:0.770732045173645
epoch:1, step:40, loss:0.7423959970474243
epoch:1, step:50, loss:0.6752623915672302
epoch:1, step:60, loss:0.6872593760490417
epoch:1, step:70, loss:0.7400232553482056
epoch:1, step:80, loss:0.7460252642631531
epoch:1, step:90, loss:0.6769644618034363
epoch:1, step:100, loss:0.6714293360710144
epoch:1, step:110, loss:0.6759071946144104
epoch:1, step:120, loss:0.6875099539756775
epoch:1, step:130, loss:0.6337903738021851
epoch:1, step:140, loss:0.762600302696228
epoch:1, step:150, loss:0.6210678815841675
epoch:1, step:160, loss:0.604053795337677
epoch:1, step:170, loss:0.646727979183197
epoch:1, step:180, loss:0.7732130289077759
epoch:1, step:190, loss:0.6990667581558228
epoch:1, step:200, loss:0.6344760060310364
epoch:1, step:210, loss:0.7843457460403442
epoch:1, step:220, loss:0.6456484794616699
epoch:1, step:230, loss:0.6415854692459106
epoch:1, step:240, loss:

In [93]:



def test():
    corrent = 0
    total = 0
    with torch.no_grad():
        for i , data in enumerate(train_loader):
            x, y, lengths = data
            y_hat = lstm(x, lengths)   

            '''
            torch.max(input, dim, keepdim=False, out=None)
            按维度dim返回最大值，0为按列取最大值，1为按行取最大值
            （最大值，最大值索引）
            下面语句代表按行取最大值对应索引，即标签值        
            prediction[0] 代表取值， [1]代表取对应的索引
            '''
            prediction = torch.max(y_hat.data, 1)[1]
            total += len(y)
            # print(total)
            corrent += (y == prediction).sum().item()
            # print(corrent)
            # break
        print('Accuracy on test set: %d %%' % (100 * corrent / total))
test()


Accuracy on test set: 33 %
